# Packages 

In [65]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import scipy
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
! ls -al ../data/candidates/ | grep w2v_task2_v2

-rw-r--r--  1 jupyter jupyter   17111651 Jun  8 10:35 task2_eval_w2v_task2_v2_True_top100.parquet
-rw-r--r--  1 jupyter jupyter   12804627 Jun  8 10:36 task2_test4task3_w2v_task2_v2_True_top100.parquet
-rw-r--r--  1 jupyter jupyter   16518343 Jun  8 10:36 task2_test_w2v_task2_v2_True_top100.parquet
drwxr-xr-x  2 jupyter jupyter       4096 Jun  7 19:08 task2_train_w2v_task2_v2_True_top100
-rw-r--r--  1 jupyter jupyter   76893690 Jun  8 10:35 task2_train_w2v_task2_v2_True_top100.parquet


# Config 

In [67]:
candidate_file_name

'{task}_{data_type}_{model_version}_{model_for_eval}_top{topn}.parquet'

In [68]:
debug = False


task = 'task1'
version = 'v10'

w2v_model_version = f'w2v_task2_v2'


vector_dim = 32
# # target locales: locales needed for task1

if task == 'task1':
    target_locals = ['DE', 'JP', 'UK']
elif task == 'task2':
    target_locals = ['ES', 'FR', 'IT']
else:
    assert 1 == 0

eval_cg = True
train_parts = 20

load_train_eval_data = False
if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None


candidate_path = '../data/candidates/'
model_dir = '../model_training'
ranker_train_data_dir = f'../data/rank_train_data_{task}_{version}'
# train_data_dir = '.'
# test_data_dir = '.'

nic_model_version = f'nic_{task}_{version}'
nfi_model_version = f'nfi_{task}_{version}'
covisit_model_version = f'co_visit_{task}_{version}'

# rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=100
nic_topn=100
nfi_topn=100
covisit_topn = 300
# PREDS_PER_SESSION = 100


# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")



train_dir = os.path.join(ranker_train_data_dir, 'train')

In [69]:
print(ranker_train_data_dir)
print(train_dir)

../data/rank_train_data_task1_v10
../data/rank_train_data_task1_v10/train


In [70]:
! mkdir {ranker_train_data_dir}

mkdir: cannot create directory ‘../data/rank_train_data_task1_v10’: File exists


In [71]:
! mkdir {train_dir}

mkdir: cannot create directory ‘../data/rank_train_data_task1_v10/train’: File exists


# Function 

In [72]:
def candidates_validation(target_pl):
    # rec_summary  = (target_pl
    #         .lazy()
    #         .select(
    #             pl.col('next_item_prediction').arr.lengths().alias('rec_num')
    #         ).collect().describe())
    # print(rec_summary)
    max_sample_per_NIP = (
        target_pl.groupby(['session_id', 'next_item_prediction'])
            .agg(
                pl.count().alias('unique_count')
                # , pl.col('w2v_weight').max().alias('max_w2v_weight')
                # , pl.col('w2v_weight').min().alias('min_w2v_weight')
            )
            .select(
                pl.col('unique_count').max().alias('max_unique_cnt')
                # , pl.col('max_w2v_weight').min().alias('min_max_w2v')
                # , pl.col('min_w2v_weight').max().alias('max_min_w2v')
            )
    ).to_pandas()['max_unique_cnt'].values[0]
    # print(stat_info)
    # print(max_sample_per_NIP)
    assert max_sample_per_NIP == 1, f"there should be unique prediction for every session_id"

## get all candidate 

In [73]:



def get_all_candidates(data_type,
                       task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version,
                       nfi_model_version=nfi_model_version,
                       covisit_model_version=covisit_model_version,
                    model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn
                       , nfi_topn=nfi_topn
                       , covisit_topn=covisit_topn
                       , part=None
                       , detail=False
                      ):
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model_version
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model_version
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    nfi_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=nfi_model_version
                , model_for_eval=model_for_eval
                , topn=nfi_topn
                       ))
    covisit_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=covisit_model_version
                , model_for_eval=model_for_eval
                , topn=covisit_topn
                       ))
    print("Candidiate files: ")
    print('+'*20)
    print()
    print(w2v_file)
    print(nic_file)
    print(nfi_file)
    print(covisit_file)
    if data_type == 'train':
        assert part is not None
        w2v_file = os.path.join(w2v_file.replace('.parquet', ''), f"part_{part}.parquet")
        nic_file = os.path.join(nic_file.replace('.parquet', ''), f"part_{part}.parquet")
        nfi_file = os.path.join(nfi_file.replace('.parquet', ''), f"part_{part}.parquet")
        covisit_file = os.path.join(covisit_file.replace('.parquet', ''), f"part_{part}.parquet")
        original_file_name = f"sessions_{data_type}.parquet"
    elif data_type == 'test':
        original_file_name = f"sessions_{data_type}_{task}.parquet"
    elif data_type == 'test4task3':
        original_file_name = f"sessions_test_task3.parquet"
    elif data_type == 'eval':
        original_file_name = f"sessions_{data_type}.parquet"
    else:
        assert 1==2

        
    original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
    w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM).filter(pl.col('rec_num')>0)#.with_columns(pl.col('prev_items').apply(str2list))
    nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM).filter(pl.col('rec_num')>0)#.with_columns(pl.col('prev_items').apply(str2list))
    nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM).filter(pl.col('rec_num')>0)#.with_columns(pl.col('prev_items').apply(str2list))
    covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM).filter(pl.col('rec_num')>0)#.with_columns(pl.col('prev_items').apply(str2list))


    if detail:
        strategies = [
            'w2v', 'nic', 'nfi', 'covisit'
        ]
        pl_lst = [w2v_pl, nic_pl, nfi_pl, covisit_pl]
        for stategy, single_pl in zip(strategies, pl_lst):
            print(f"Type: {stategy}")
            # print(single_pl.schema)
            stat_info = (
                single_pl
                    .select(
                        pl.col("next_item_prediction").arr.lengths().alias("rec_cnt")
                    )
                    .collect()
                    .describe()
                    
            )
            print(stat_info)
            # null_case = single_pl.explode('next_item_prediction').filter(pl.col('next_item_prediction').is_null()).head().collect()
            # print(null_case)
        print()
        print()
    
    # get w2v weight
    w2v_pl = (
        w2v_pl
        .with_columns(
            pl.col('next_item_prediction').arr.lengths().alias('rec_num'))
        .with_columns(
                pl.col('rec_num').apply(lambda x: list(range(x, 0, -1))).alias('w2v_weight')
                , pl.col('rec_num').apply(lambda x: list([1]*x)).alias('whether_w2v')
                , pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_nic')
                , pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_nfi')
                , pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_covisit')
        )
    )
    # print(w2v_pl.head().collect())
    get_w2v_weight = pl.element().rank()*0
    nic_pl = (
        nic_pl
            .with_columns(
                pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                                        # parallel=True
                                                       ).alias('w2v_weight').cast(pl.List(pl.Int64))
            )
        .with_columns(
            pl.col('next_item_prediction').arr.lengths().alias('rec_num'))
        .with_columns(
                pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_w2v')
                , pl.col('rec_num').apply(lambda x: list([1]*x)).alias('whether_nic')
                , pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_nfi')
                , pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_covisit')

        )
    )
    nfi_pl = (
        nfi_pl
        .with_columns(
            pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                                   ).alias('w2v_weight').cast(pl.List(pl.Int64))
        )
        .with_columns(
            pl.col('next_item_prediction').arr.lengths().alias('rec_num'))
        .with_columns(
                pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_w2v')
                , pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_nic')
                , pl.col('rec_num').apply(lambda x: list([1]*x)).alias('whether_nfi')
                , pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_covisit')

        )
    )
    covisit_pl = (
        covisit_pl.with_columns(
            pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                                   ).alias('w2v_weight').cast(pl.List(pl.Int64))
        )
        .with_columns(
            pl.col('next_item_prediction').arr.lengths().alias('rec_num'))
        .with_columns(
                pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_w2v')
                , pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_nic')
                , pl.col('rec_num').apply(lambda x: list([0]*x)).alias('whether_nfi')
                , pl.col('rec_num').apply(lambda x: list([1]*x)).alias('whether_covisit')

        )
    )
    cols = ['session_id', 'next_item_prediction', 
            'w2v_weight', 'whether_w2v','whether_nic', 'whether_nfi', 'whether_covisit'
           ]
    # combined_pl = (
    #     w2v_pl.select(cols)
    #         .join(nic_pl.select(cols), how='left', on='session_id', suffix='_nic')
    #         .join(nfi_pl.select(cols), how='left', on='session_id', suffix='_nfi')
    # )

    # combined_pl = combined_pl.with_columns(
    #     pl.concat_list([pl.col('next_item_prediction'), pl.col('next_item_prediction_nic')]).alias('next_item_prediction'), 
    #     pl.concat_list([pl.col('w2v_weight'), pl.col('w2v_weight_nic')]).alias('w2v_weight'), 
    # ).select(
    #     pl.all().exclude(['next_item_prediction_nic', 'w2v_weight_nic'])
    # )
    explode_cols = ['next_item_prediction', 
                    'w2v_weight', 'whether_w2v', 'whether_nic', 'whether_nfi', 'whether_covisit'
                   ]
    # pl_lst = []
    # strategies_cols = [
    #     'whether_w2v'
    #     , 'whether_nic'
    #     , 'whether_nfi'
    #     , 'whether_covisit'
    # ]
    # strategies = []
    
#     w2v_pl = w2v_pl.with_columns(
#         pl.lit(1).alias('whether_w2v')
#     )
#     nic_pl = w2v_pl.with_columns(
#         pl.lit(0).alias('whether_w2v')
#     )
#     nfi_pl = w2v_pl.with_columns(
#         pl.lit(0).alias('whether_w2v')
#     )
#     covisit_pl = w2v_pl.with_columns(
#         pl.lit(0).alias('whether_w2v')
#     )
    
    combined_pl = (
            pl.concat([w2v_pl.select(cols).explode(explode_cols).filter(~pl.col('next_item_prediction').is_null())
                       , nic_pl.select(cols).explode(explode_cols).filter(~pl.col('next_item_prediction').is_null())
                       , nfi_pl.select(cols).explode(explode_cols).filter(~pl.col('next_item_prediction').is_null())
                       , covisit_pl.select(cols).explode(explode_cols).filter(~pl.col('next_item_prediction').is_null())
                      ], how='vertical')
                .groupby(['session_id', 'next_item_prediction'])
                .agg(
                    pl.col('w2v_weight').max().alias('w2v_weight')
                    , pl.col('whether_w2v').max().alias('whether_w2v')
                    , pl.col('whether_nic').max().alias('whether_nic')
                    , pl.col('whether_nfi').max().alias('whether_nfi')
                    , pl.col('whether_covisit').max().alias('whether_covisit')

                )
                .with_columns(
                    pl.col('whether_w2v')
                    .add(pl.col('whether_nic'))
                    .add(pl.col('whether_nfi'))
                    .add(pl.col('whether_covisit'))
                    .alias('strategy_num')
                )
                .join(original_pl.select(['session_id', 'prev_items']), how='left', on='session_id')
                .filter(pl.col('prev_items').arr.contains(pl.col('next_item_prediction'))==False)
                .with_columns(
                    pl.col('w2v_weight').max().over('session_id').alias('max_count')
                    , pl.col('w2v_weight').min().over('session_id').alias('min_count')
                )
                .with_columns(
                    pl.when(pl.col('max_count')==pl.col('min_count')).then(1).otherwise((pl.col('w2v_weight')-pl.col('min_count'))/(pl.col('max_count')-pl.col('min_count'))).alias('w2v_weight')
                )
                .select(
                        pl.exclude(['prev_items', 'max_count', 'min_count'])

                )
                .groupby('session_id')
                .agg(
                    pl.all()
                )
    )
    # if include_w2v_vector:
        
    combined_pl = (
        combined_pl.join(original_pl, how='left', on='session_id')
            .with_columns(
                pl.col('prev_items').arr.lengths().alias('prev_length')
                , pl.col('prev_items').arr.get(-1).alias('last_prev_item')
                , pl.col('next_item_prediction').arr.lengths().alias('rec_num')
            ).select(
                pl.all().exclude('prev_items')
            )
    )
    return combined_pl


In [74]:
# %%time

# part_index=0
# get_all_candidates(data_type='train', part=part_index+1, 
#                    # detail=True
#                                     ).head().collect()

In [75]:
# %%time

# eval_cg_pl = get_all_candidates(data_type='eval', detail=False)


In [76]:
# eval_cg_pl.head().collect()

In [77]:
# eval_cg_pl.explode('strategy_num').select('strategy_num').collect().to_series().value_counts()

In [78]:
# (
#     eval_cg_pl
#         .select(['session_id', 'strategy_num']) 
#         .explode('strategy_num')
#         .with_columns(
#             pl.when(pl.col('strategy_num')>=4).then(1).otherwise(0).alias('4_num')
#             ,pl.when(pl.col('strategy_num')>=3).then(1).otherwise(0).alias('3_num')
#             ,pl.when(pl.col('strategy_num')>=2).then(1).otherwise(0).alias('2_num')
#             ,pl.when(pl.col('strategy_num')>=1).then(1).otherwise(0).alias('1_num')

#         )
#         .groupby('session_id')
#         .agg(
#             pl.col('4_num').sum()
#             , pl.col('3_num').sum()
#             , pl.col('2_num').sum()
#             , pl.col('1_num').sum()

#         )
#         .collect()
#         .describe()
# )
# # .head().collect().describe()

In [79]:
1/(11+12/10+100/20)

0.05813953488372093

In [80]:
11+12/10+100/20

17.2

In [81]:
# 20

In [82]:
# eval_cg_pl.head().collect()

In [83]:
# eval_cg_pl.explode('strategy_num').filter(pl.col('strategy_num').is_null()).head().collect()

In [84]:
# %%time

# part_index=0
# get_all_candidates(data_type='train', part=part_index+1, 
#                    # detail=True
#                                     ).head().collect()

## Get features

In [85]:
# eval_cg_pl.schema

In [86]:
# case

In [87]:
# target_df = train_candidates
# data_type = 'train'

def get_w2v_simi(x, col='last_prev_item'):
    try:
        if isinstance(x[col], str):
            target = [x[col]]
        else:
            target = x[col]
        simi = w2vec.wv.n_similarity([x['next_item_prediction']],
                                                  target
                                                 )
    except Exception as e:
        print(e)
        # print(x[col])
        simi = 0
    return simi

def get_vector(item_id, vector_dim=32):
    try:
        vector = w2vec.wv[item_id].tolist()[:vector_dim]
        # print(vector)
        return vector
        # return [0.0]*vector_dim
    except:
        return [0.0]*vector_dim

def get_feature(target_df, data_type
                , nic_model#=nic_model
                , nfi_model#=nfi_model
                , co_visit_model#=co_visit_model
                , negative_sampling=None, w2v_similarity=False
               , min_sample_num=20
                , include_w2v_vector=False
               ):
    if data_type == 'train':
        # remove data where recall@all>0
        target_df = target_df.filter(pl.col('next_item_prediction').arr.contains(pl.col('next_item')))
    target_df = target_df.explode(['next_item_prediction', 'w2v_weight', 'strategy_num'
                              , 'whether_w2v', 'whether_nic', 'whether_nfi', 'whether_covisit'])

    if data_type not in ['test', 'test4task3']:
        # add target label for the data
        target_df = (
            target_df
                .with_columns(
                    pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
                )
        )

    if negative_sampling is not None:
        minor = target_df.filter(pl.col('rec_num')<=min_sample_num)
        major = target_df.filter(pl.col('rec_num')>min_sample_num)
        cols = ['session_id', 'next_item', 'next_item_prediction', 'target', 'strategy_num'
                              , 'whether_w2v', 'whether_nic', 'whether_nfi', 'whether_covisit']
        sample_pl = major.select(cols)
        pos = sample_pl.filter(pl.col('target')==1)
        neg = (sample_pl
                   .filter(pl.col('target')==0)
              )
        neg_original_len = neg.select('session_id').collect().shape[0]
        print(f'Original negative num: {neg_original_len}')
        # neg = (
        #     neg.with_columns(
        #                 pl.Series(name='random', values=np.random.uniform(size=neg_original_len))
        #            )
        #            .filter(pl.col('random')<=negative_sampling)
        #            .select(pl.all().exclude('random'))
        # )
        
        neg_4_3 = neg.filter(pl.col('strategy_num')>=3).groupby('session_id').agg(
                pl.all()
            )
        neg_2 = neg.filter(pl.col('strategy_num')==2).groupby('session_id').agg(
                pl.all().sample(frac=0.1)
            )
        neg_1 = neg.filter(pl.col('strategy_num')==1).groupby('session_id').agg(
                pl.all().sample(frac=0.05)
            )
        # neg = neg.groupby('session_id').agg(
        #     pl.all().sample(frac=negative_sampling)
        #     )
        # print(neg_4_3.schema())
        # print(neg_4_3.head().collect())
        neg = pl.concat([neg_4_3, neg_2, neg_1], how='vertical')
        neg = neg.explode(neg.columns[1:])
        # # print(neg.shape)
        # # print(neg.head().collect())
        sample_pl = pl.concat([pos, neg], how='vertical')
        
        majoy = sample_pl.join(major, how='left', on=cols)#.collect()
        target_pl = pl.concat([major, minor], how='vertical')
        del sample_pl
    target_df = (
            target_df.lazy()
                .join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
                .join(nfi_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_nfi')
                .join(co_visit_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_co_visit')
                .with_columns(
                    pl.struct(["next_item_prediction", "last_prev_item"]).apply(
                        lambda x: get_w2v_simi(x, col='last_prev_item')).alias('last_item_similarity').cast(pl.Float32)
                    # , pl.struct(["next_item_prediction", "prev_items"]).apply(
                    #     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
                    , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                        .then(2)
                        .otherwise(3).alias('locale')
                ).with_columns(
                    pl.all().fill_null(0.0)
                )
                .sort('session_id')
                .select(
                    pl.all().exclude(['last_prev_item'])
                )
    )
    if include_w2v_vector:
        target_df = target_df.with_columns(
                    pl.col('next_item_prediction').apply(lambda s: get_vector(item_id=s)).alias('w2v_vec'),
                ).with_columns(
                    [pl.col("w2v_vec").arr.get(i).alias(f"vec_{i}") for i in range(vector_dim)]
                ).select(
                    pl.all().exclude(['w2v_vec'])
                )
    # if w2v_similarity:
    #     target_df = target_df.with_columns(
    #         pl.struct(["next_item_prediction", "prev_items"]).apply(
    #                     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
    #     )
    return target_df
# target_df.head(3).collect()

# Load model 

In [88]:
nic_model = (
    pl.scan_parquet(f'../model_training/{nic_model_version}/nic_True_for_eval.parquet')
        .explode(['next_item_prediction', 'next_item_weight', 'next_item_cnt'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
        )
            )
print(nic_model.schema)

nfi_model = (
    pl.scan_parquet(f'../model_training/{nfi_model_version}/nic_True_for_eval.parquet')
        .explode(['next_item_prediction', 'next_item_weight', 'next_item_normalized_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
            # , pl.col()
        )
            )
print(nfi_model.schema)



w2vec = Word2Vec.load(w2v_model_file)
annoy_index = AnnoyIndexer()
annoy_index.load(annoy_index_file)


co_visit_model = (
    pl.scan_parquet(f'../model_training/{covisit_model_version}/{covisit_model_version}_True_for_eval.parquet')
        .explode(['next_item_prediction', 'next_item_weight'
                  , 'discount_weight' # co-visit discount weight
         , 'following_weight' 
         , 'following_discount_weight'
            ,'previous_weight'
            , 'previous_discount_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
            # , pl.col()
        )
            )
print(co_visit_model.schema)

{'next_item_prediction': Utf8, 'next_item_weight': Float64, 'next_item_cnt': UInt32, 'last_prev_item': Utf8}
{'next_item_prediction': Utf8, 'next_item_weight': Float32, 'next_item_normalized_weight': Float64, 'last_prev_item': Utf8}
{'next_item_prediction': Utf8, 'next_item_weight': Float32, 'discount_weight': Float32, 'following_weight': Float32, 'following_discount_weight': Float32, 'previous_weight': Float32, 'previous_discount_weight': Float32, 'last_prev_item': Utf8}


# W2V features

In [89]:
# debug_pl = pl.scan_parquet(os.path.join('../data/raw_data_session_id/', 'sessions_train1.parquet'), n_rows=100).with_columns(pl.col('prev_items').apply(str2list)).collect()


In [29]:
# debug_pl.head()

In [30]:
# (
#  pl.DataFrame({'a': ["1 2 3 4 5 6"]})
#     .select([pl.col("a").str.split(" ").arr.get(i).alias(f"col{i}") for i in range(6)])
# # shape: (1, 6)
# )

In [31]:
# vector_dim = 32
# (
#     debug_pl
#         .with_columns(
#             pl.col('next_item').apply(lambda s: get_vector(item_id=s)).alias('w2v_vec'),
#         )
#         .with_columns(
#             [pl.col("w2v_vec").arr.get(i).alias(f"vec_{i}") for i in range(vector_dim)]
#         )
# )

In [32]:
# w2vec.wv['B08YNZT93Z'].tolist()

# Re-process data for memmory 

In [33]:
def get_train_candidate(
    task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version,
                       nfi_model_version=nfi_model_version,
                       covisit_model_version=covisit_model_version,
                    model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn
                       , nfi_topn=nfi_topn
                       , covisit_topn=covisit_topn
                    , train_parts=train_parts
):
    data_type = 'train'
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model_version
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model_version
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    nfi_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=nfi_model_version
                , model_for_eval=model_for_eval
                , topn=nfi_topn
                       ))
    covisit_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=covisit_model_version
                , model_for_eval=model_for_eval
                , topn=covisit_topn
                       ))
    # w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    dir_lst = []
    pl_lst = []
    for file_name in [w2v_file, nic_file, nfi_file, covisit_file]:
        data_pl = pl.scan_parquet(file_name, n_rows=SAMPLE_NUM).collect().sort('session_id')
        dir_name = file_name.replace('.parquet', '')
        print(dir_name)
        dir_lst.append(dir_name)
        pl_lst.append(data_pl)
        if not os.path.isdir(dir_name):
            os.mkdir(dir_name)
    # assert pl_lst[0].shape[0] == pl_lst[1].shape[0] == pl_lst[2].shape[0] == pl_lst[3].shape[0], f"{[ele.shape for ele in pl_lst]}"
    
    
    # get all of the session_id
    cols_to_keep = 'session_id'
    session_id_pl = (
        pl.concat([ele.select(cols_to_keep) for ele in pl_lst], how='vertical')
            .groupby('session_id')
            .agg(pl.count())
            .select('session_id')
    )
    # -------------------
    return pl_lst, dir_lst, session_id_pl

def re_process_data(task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version,
                       nfi_model_version=nfi_model_version,
                       covisit_model_version=covisit_model_version,
                    model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn
                       , nfi_topn=nfi_topn
                       , covisit_topn=covisit_topn
                    , train_parts=train_parts
                      ):
    parts = train_parts
    pl_lst, dir_lst, session_id_pl= get_train_candidate(
            task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version,
                       nfi_model_version=nfi_model_version,
                       covisit_model_version=covisit_model_version,
                    model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn
                       , nfi_topn=nfi_topn
                       , covisit_topn=covisit_topn
                    , train_parts=parts
    )
    total_len = session_id_pl.shape[0]

    rows_per_part = int(total_len/parts)
    for idx in tqdm(range(parts)):
        save_shape_lst = []
        save_after_lst = []
        for dir_path, data_pl in zip(dir_lst, pl_lst):
            # print(dir_path)
            begin = idx*rows_per_part
            if idx != parts-1:
                end = (idx+1)*rows_per_part
            else:
                end = total_len
            part_session_pl = session_id_pl[begin:end :]
            part_pl = part_session_pl.join(data_pl, how='left', on='session_id')
            file_name = os.path.join(dir_path, f"part_{idx+1}.parquet")
            print(f'{begin} - {end} -> {file_name}')
            save_shape_lst.append(part_pl.shape[0])
            # print(part_pl.head())
            part_pl = part_pl.filter(~pl.col('next_item_prediction').is_null())
            save_after_lst.append(part_pl.shape[0])
            part_pl.write_parquet(file_name)
            
        print(f"{save_shape_lst} -> {save_after_lst}")
    

In [34]:
# pl

In [35]:
reprocess_train_data = False

In [36]:
task

'task1'

In [37]:
train_parts

20

In [38]:
if reprocess_train_data:
    re_process_data(w2v_model_version=f"w2v_{task}_{version}")

# Get candiadtes

In [39]:
# data_type = 'eval'
# w2v_file = os.path.join(candidate_path, 
#                            candidate_file_name.format(
#                     task=task
#                 , data_type=data_type
#                 , model_version=w2v_model_version
#                 , model_for_eval=model_for_eval
#                 , topn=w2v_topn
#                            ))
# w2v_file
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)

In [40]:
    # w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))


In [44]:
eval_cg_pl = get_all_candidates(data_type='eval',
                               w2v_model_version=f'w2v_{task}_{version}')


Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_eval_w2v_task1_v10_True_top100.parquet
../data/candidates/task1_eval_nic_task1_v10_True_top100.parquet
../data/candidates/task1_eval_nfi_task1_v10_True_top100.parquet
../data/candidates/task1_eval_co_visit_task1_v10_True_top300.parquet


In [45]:
# eval_cg_pl.head().collect()

In [42]:
# ! ls ../data/candidates/ | grep test

In [51]:
test_cg_pl = get_all_candidates(data_type='test', w2v_model_version=f'w2v_{task}_{version}')

Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_test_w2v_task1_v10_True_top100.parquet
../data/candidates/task1_test_nic_task1_v10_True_top100.parquet
../data/candidates/task1_test_nfi_task1_v10_True_top100.parquet
../data/candidates/task1_test_co_visit_task1_v10_True_top300.parquet


In [52]:
# eval_cg_pl.select('rec_num').collect().to_pandas().describe([0.1*i for i in range(10)])

In [53]:
# test_cg_pl.select('rec_num').collect().to_pandas().describe([0.1*i for i in range(10)])

In [41]:
# eval_cg_pl.head().collect()

In [42]:
# eval_cg_pl.select(
#     pl.col('next_item_prediction').arr.lengths().alias('len1')
#     , pl.col('w2v_weight').arr.lengths().alias('len2')
# ).collect().describe()

In [43]:
# eval_cg_pl.select(
#     pl.col('next_item_prediction').arr.lengths().alias('len1')
#     , pl.col('w2v_weight').arr.lengths().alias('len2')
# ).collect().describe()

In [44]:
# ).collect().describe()

In [45]:
# train_cg_pl.head().collect()

In [107]:
# eval_cg_pl.head().collect()

In [46]:
# eval_cg_pl.head().collect()

# Eval candidate generation 

In [47]:
if eval_cg:
    eval_topn = 3000

    col = f"recall@{eval_topn}"
    eval_final = (
            eval_cg_pl
            .lazy()
            .with_columns(
                pl.col('next_item_prediction').cast(pl.List(pl.Utf8))
            )
            ).with_columns(
                pl.col('next_item_prediction').arr.head(eval_topn).arr.contains(pl.col('next_item')).mean().alias(col)

            )
    final_res = eval_final.select(
            pl.count().alias('total_sessions')
            , pl.col(col).mean()

        ).collect()
    describe = (eval_cg_pl
            .lazy()
            .select(
                pl.col('next_item_prediction').arr.lengths().alias('rec_num')
            ).collect().describe())
    print(describe)
    print(final_res)
    del final_res

shape: (7, 2)
┌────────────┬────────────┐
│ describe   ┆ rec_num    │
│ ---        ┆ ---        │
│ str        ┆ f64        │
╞════════════╪════════════╡
│ count      ┆ 321029.0   │
│ null_count ┆ 0.0        │
│ mean       ┆ 177.316224 │
│ std        ┆ 109.206794 │
│ min        ┆ 1.0        │
│ max        ┆ 478.0      │
│ median     ┆ 146.0      │
└────────────┴────────────┘
shape: (1, 2)
┌────────────────┬─────────────┐
│ total_sessions ┆ recall@3000 │
│ ---            ┆ ---         │
│ u32            ┆ f64         │
╞════════════════╪═════════════╡
│ 321029         ┆ 0.597211    │
└────────────────┴─────────────┘


In [48]:
# co_visit_model.head().collect()

In [49]:
# debug_pl.head()

In [50]:
# %%time
# sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train',
#                                        negative_sampling=0.1,
#                                       w2v_similarity=True)
# debug_pl = sampled_train_candidates.head().collect()

In [51]:
# debug_pl

In [52]:
# prev_items = ["B09W9FND7K", "B09JSPLN1M"]
# next_item_prediction = "B09M7GY217"	
# w2vec.wv.n_similarity(prev_items, [next_item_prediction])
# w2vec.wv.n_similarity(prev_items, prev_items)
# prev_items_vec = np.array([w2vec.wv[ele] for ele in prev_items]).mean(axis=0)
# next_item_vec = w2vec.wv[next_item_prediction]
# 1 - scipy.spatial.distance.cosine(prev_items_vec, next_item_vec)

# Save Data 

## Save eval data 

In [53]:
# eval_cg_pl

In [46]:
%load_ext memory_profiler

In [47]:
eval_candidates = get_feature(target_df=eval_cg_pl, data_type='eval'
                             , nic_model=nic_model
                             , nfi_model=nfi_model
                              , co_visit_model=co_visit_model
                             )

In [ ]:
# eval_candidates.head().collect()

In [ ]:
%%time
%memit
eval_candidates = eval_candidates.collect()

print('Eval data')
print()
print(eval_candidates.shape)
print(f"Memory: {eval_candidates.estimated_size(unit='gb')}")
print(eval_candidates['target'].mean())
print('Validating')
candidates_validation(target_pl=eval_candidates)
print()

if not debug:
    eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))

peak memory: 2347.93 MiB, increment: 0.00 MiB
Eval data

(56923650, 24)
Memory: 9.54257046431303
0.0033680552810650757
Validating

CPU times: user 38min 26s, sys: 1min 50s, total: 40min 16s
Wall time: 27min 23s


In [ ]:
del eval_candidates

## test data

In [ ]:
# test_cg_pl.explode('next_item_prediction').filter(pl.col('next_item_prediction').is_null()).head().collect()

In [ ]:
# eval_cg_pl.explode('next_item_prediction').filter(pl.col('next_item_prediction').is_null()).head().collect()

In [ ]:
# train_cg_pl.explode('next_item_prediction').filter(pl.col('next_item_prediction').is_null()).head().collect()

In [ ]:
# train_cg_pl.head().collect()

In [54]:
test_candidates = get_feature(target_df=test_cg_pl, data_type='test'
                            , nic_model=nic_model
                             , nfi_model=nfi_model
                              , co_visit_model=co_visit_model
                             )

In [55]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [56]:
ranker_train_data_dir

'../data/rank_train_data_task1_v10'

In [57]:
%%time
%memit

test_candidates = test_candidates.collect()
print('Test data')
print()
print(test_candidates.shape)
print(f"Memory: {test_candidates.estimated_size(unit='gb')}")
print('Validating')
candidates_validation(target_pl=test_candidates)

if not debug:
    # eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))
    test_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'))

peak memory: 11565.27 MiB, increment: 0.00 MiB
Test data

(68797218, 22)
Memory: 9.867149945348501
Validating
CPU times: user 44min 15s, sys: 2min, total: 46min 15s
Wall time: 32min 47s


In [58]:
del test_candidates

In [59]:
# eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))
# test_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'))

## Saving test4task3 

In [60]:
test4task3_cg_pl = get_all_candidates(data_type='test4task3'
                                      , w2v_model_version=f'w2v_{task}_{version}'
                                                                 
                                     )

Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_test4task3_w2v_task1_v10_True_top100.parquet
../data/candidates/task1_test4task3_nic_task1_v10_True_top100.parquet
../data/candidates/task1_test4task3_nfi_task1_v10_True_top100.parquet
../data/candidates/task1_test4task3_co_visit_task1_v10_True_top300.parquet


In [61]:
test4task3_candidates = get_feature(target_df=test4task3_cg_pl, data_type='test4task3'
                                   , nic_model=nic_model
                             , nfi_model=nfi_model
                              , co_visit_model=co_visit_model)
test4task3_candidates.collect().write_parquet(os.path.join(ranker_train_data_dir, 'test4task3.parquet'))

In [63]:
del test4task3_candidates

## Saving train data 

In [ ]:
# train_cg_pl.collect().sample(10)

In [ ]:
# part_index = 0

In [ ]:
# train_cg_pl.head().collect()

In [ ]:
# print()
# file_name = os.path.join(train_dir, f"part_{part_index+1}.parquet")
# print(file_name)
# train_cg_pl = get_all_candidates(data_type='train', part=part_index+1, 
#                                  # original_file_name='../data/raw_data_session_id/sess'
#                                 )

In [ ]:
# target_df=train_cg_pl
# data_type='train'
# negative_sampling=0.1
# # nic_model=nic_model
# #  nfi_model=nfi_model
# #                           , co_visit_model=co_visit_model

In [ ]:

# if data_type == 'train':
#     # remove data where recall@all>0
#     target_df = target_df.filter(pl.col('next_item_prediction').arr.contains(pl.col('next_item')))
# target_df = target_df.explode(['next_item_prediction', 'w2v_weight', 'strategy_num'
#                           , 'whether_w2v', 'whether_nic', 'whether_nfi', 'whether_covisit'])

# if data_type not in ['test', 'test4task3']:
#     # add target label for the data
#     target_df = (
#         target_df
#             .with_columns(
#                 pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
#             )
#     )

# # if negative_sampling is not None:
# #     minor = target_df.filter(pl.col('rec_num')<=min_sample_num)
# #     major = target_df.filter(pl.col('rec_num')>min_sample_num)
# #     cols = ['session_id', 'next_item', 'next_item_prediction', 'target', 'strategy_num'
# #                           , 'whether_w2v', 'whether_nic', 'whether_nfi', 'whether_covisit']
# #     sample_pl = major.select(cols)
# #     pos = sample_pl.filter(pl.col('target')==1)
# #     neg = (sample_pl
# #                .filter(pl.col('target')==0)
# #           )
# #     neg_original_len = neg.select('session_id').collect().shape[0]
# #     print(f'Original negative num: {neg_original_len}')
# #     # neg = (
# #     #     neg.with_columns(
# #     #                 pl.Series(name='random', values=np.random.uniform(size=neg_original_len))
# #     #            )
# #     #            .filter(pl.col('random')<=negative_sampling)
# #     #            .select(pl.all().exclude('random'))
# #     # )

# #     neg_4_3 = neg.filter(pl.col('strategy_num')>=3).groupby('session_id').agg(
# #             pl.all()
# #         )
# #     neg_2 = neg.filter(pl.col('strategy_num')==2).groupby('session_id').agg(
# #             pl.all().sample(frac=0.1)
# #         )
# #     neg_1 = neg.filter(pl.col('strategy_num')==1).groupby('session_id').agg(
# #             pl.all().sample(frac=0.05)
# #         )
# #     # neg = neg.groupby('session_id').agg(
# #     #     pl.all().sample(frac=negative_sampling)
# #     #     )
# #     # print(neg_4_3.schema())
# #     # print(neg_4_3.head().collect())
# #     neg = pl.concat([neg_4_3, neg_2, neg_1], how='vertical')
# #     neg = neg.explode(neg.columns[1:])
# #     # # print(neg.shape)
# #     # # print(neg.head().collect())
# #     sample_pl = pl.concat([pos, neg], how='vertical')

# #     majoy = sample_pl.join(major, how='left', on=cols)#.collect()
# #     target_pl = pl.concat([major, minor], how='vertical')
# #     del sample_pl
# # target_df = (
# #         target_df.lazy()
# #             .join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
# #             .join(nfi_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_nfi')
# #             .join(co_visit_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_co_visit')
# #             .with_columns(
# #                 # pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
# #                 # , pl.when(pl.col('next_item_weight_nfi').is_null()).then(0).otherwise(pl.col('next_item_weight_nfi')).alias('next_item_weight_nfi')
# #                 # , pl.when(pl.col('next_item_weight_co_visit').is_null()).then(0).otherwise(pl.col('next_item_weight_co_visit')).alias('next_item_weight_co_visit')


# #                 pl.struct(["next_item_prediction", "last_prev_item"]).apply(
# #                     lambda x: get_w2v_simi(x, col='last_prev_item')).alias('last_item_similarity').cast(pl.Float32)
# #                 # , pl.struct(["next_item_prediction", "prev_items"]).apply(
# #                 #     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
# #                 , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
# #                     .then(2)
# #                     .otherwise(3).alias('locale')
# #             )
# #             .with_columns(
# #                 pl.all().fill_null(0.0)
# #             )
# #             .sort('session_id')
# #             .select(
# #                 pl.all().exclude('last_prev_item')
# #             )

# # )
# # # if w2v_similarity:
# # #     target_df = target_df.with_columns(
# # #         pl.struct(["next_item_prediction", "prev_items"]).apply(
# # #                     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
# # #     )
# # return target_df
# # # target_df.head(3).collect()

In [ ]:
# neg_original_len = neg.select('session_id').collect().shape[0]


In [ ]:
# print(f'Original negative num: {neg_original_len}')
# # neg = (
# #     neg.with_columns(
# #                 pl.Series(name='random', values=np.random.uniform(size=neg_original_len))
# #            )
# #            .filter(pl.col('random')<=negative_sampling)
# #            .select(pl.all().exclude('random'))
# # )

# neg_4_3 = neg.filter(pl.col('strategy_num')>=3).groupby('session_id').agg(
#         pl.all()
#     )
# neg_2 = neg.filter(pl.col('strategy_num')==2).groupby('session_id').agg(
#         pl.all().sample(frac=0.1)
#     )
# neg_1 = neg.filter(pl.col('strategy_num')==1).groupby('session_id').agg(
#         pl.all().sample(frac=0.05)
#     )
# # neg = neg.groupby('session_id').agg(
# #     pl.all().sample(frac=negative_sampling)
# #     )


In [ ]:
# neg_4_3.head().collect()

In [ ]:
# neg_4_3.hea

In [ ]:
# # print(neg_4_3.schema())
# # print(neg_4_3.head().collect())
# neg = pl.concat([neg_4_3, neg_2, neg_1], how='vertical')
# neg = neg.explode(neg.columns[1:])
# # # print(neg.shape)
# # # print(neg.head().collect())
# sample_pl = pl.concat([pos, neg], how='vertical')
# target_df = sample_pl.join(target_df, how='left', on=cols)#.collect()
# del sample_pl
# target_df = (
#     target_df.lazy()
#         .join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
#         .join(nfi_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_nfi')
#         .join(co_visit_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_co_visit')
#         .with_columns(
#             # pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
#             # , pl.when(pl.col('next_item_weight_nfi').is_null()).then(0).otherwise(pl.col('next_item_weight_nfi')).alias('next_item_weight_nfi')
#             # , pl.when(pl.col('next_item_weight_co_visit').is_null()).then(0).otherwise(pl.col('next_item_weight_co_visit')).alias('next_item_weight_co_visit')


#             pl.struct(["next_item_prediction", "last_prev_item"]).apply(
#                 lambda x: get_w2v_simi(x, col='last_prev_item')).alias('last_item_similarity').cast(pl.Float32)
#             # , pl.struct(["next_item_prediction", "prev_items"]).apply(
#             #     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
#             , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
#                 .then(2)
#                 .otherwise(3).alias('locale')
#         )
#         .with_columns(
#             pl.all().fill_null(0.0)
#         )
#         .sort('session_id')
#         .select(
#             pl.all().exclude('last_prev_item')
#         )

# )
# # if w2v_similarity:
# #     target_df = target_df.with_columns(
# #         pl.struct(["next_item_prediction", "prev_items"]).apply(
# #                     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
# #     )
# # return target_df
# # target_df.head(3).collect()

In [ ]:
# target_df.head().collect()

In [ ]:
# sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', 
#                                        negative_sampling=0.1
#                             , nic_model=nic_model
#                          , nfi_model=nfi_model
#                           , co_visit_model=co_visit_model
#                                       )
# sampled_train_candidates = sampled_train_candidates.collect()


In [ ]:

# print('Validating')
# candidates_validation(target_pl=sampled_train_candidates)
# # print('Sampled train data')
# print(sampled_train_candidates.shape)
# print(f"Memory: {sampled_train_candidates.estimated_size(unit='gb')}")
# print(sampled_train_candidates['target'].mean())

# stat_info = (
#         sampled_train_candidates
#             .groupby('session_id')
#             .agg(
#                 pl.count().alias('rec_cnt')
#                 , pl.col('target').sum().alias('target_num')
#             )
#             .describe()
#     )
# print(stat_info)
# if not debug:
#     sampled_train_candidates.write_parquet(file_name)

In [ ]:
# sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', 
#                                        negative_sampling=0.1
#                             , nic_model=nic_model
#                          , nfi_model=nfi_model
#                           , co_visit_model=co_visit_model
#                                       )

In [ ]:
# sampled_train_candidates.head()

In [ ]:
# sampled_train_candidates = sampled_train_candidates.filter(pl.col('next_item_prediction')!='0.0')

In [ ]:
# sampled_train_candidates = sampled_train_candidates.collect()
# print('Validating')
# candidates_validation(target_pl=sampled_train_candidates)

In [ ]:
# part_index = 0

In [ ]:
# stat_info = (
#         sampled_train_candidates
#             .groupby('session_id')
#             .agg(
#                 pl.count().alias('rec_cnt')
#                 , pl.col('target').sum().alias('target_num')
#             )
#             .describe()
#     )
# print(stat_info)

In [ ]:
# train_cg_pl.filter(pl.col('rec_num')==1).head().collect()

In [ ]:
# train_cg_pl.select('rec_num').collect().to_pandas().describe([0.1*i for i in range(10)])

In [ ]:
# (sampled_train_candidates
#     .groupby('session_id')
#     .agg(
#         pl.count().alias('rec_cnt')
#         , pl.col('target').sum().alias('target_num')
#     )
#      .filter(pl.col('rec_cnt')<=1)
# )

In [ ]:
# print()
# file_name = os.path.join(train_dir, f"part_{part_index+1}.parquet")
# print(file_name)
# train_cg_pl = get_all_candidates(data_type='train', part=part_index+1, 
#                                  # original_file_name='../data/raw_data_session_id/sess'
#                                 )
# sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', 
#                                        negative_sampling=0.1
#                             , nic_model=nic_model
#                          , nfi_model=nfi_model
#                           , co_visit_model=co_visit_model
#                                       )
# sampled_train_candidates = sampled_train_candidates.collect()
# sampled_train_candidates = sampled_train_candidates.filter(pl.col('next_item_prediction')!='0.0')
# print('Validating')
# candidates_validation(target_pl=sampled_train_candidates)
# # print('Sampled train data')
# print(sampled_train_candidates.shape)
# print(f"Memory: {sampled_train_candidates.estimated_size(unit='gb')}")
# print(sampled_train_candidates['target'].mean())

# stat_info = (
#         sampled_train_candidates
#             .groupby('session_id')
#             .agg(
#                 pl.count().alias('rec_cnt')
#                 , pl.col('target').sum().alias('target_num')
#             )
#             .describe(percentiles=[0.1*i for i in range(10)])
#     )
# print(stat_info)
# if not debug:
#     sampled_train_candidates.write_parquet(file_name)
# # del sampled_train_candidates

In [ ]:
# file_name

In [ ]:
! ls ../data/rank_train_data_task2_v2/train/

In [ ]:
# ! rm ../data/rank_train_data_task2_v2/train/*

In [ ]:
! ls ../data/{ranker_train_data_dir}

In [ ]:
if debug:
    train_parts = 1
for part_index in tqdm(range(train_parts)):
    file_name = os.path.join(train_dir, f"part_{part_index+1}.parquet")
    print(file_name)
    # print(file_name)
    train_cg_pl = get_all_candidates(data_type='train', 
                                     part=part_index+1,
                                     # original_file_name='../data/raw_data_session_id/sess'
                                     w2v_model_version=f'w2v_{task}_{version}'
                                    )
    sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', 
                                           negative_sampling=0.1
                                , nic_model=nic_model
                             , nfi_model=nfi_model
                              , co_visit_model=co_visit_model
                                          )
    sampled_train_candidates = sampled_train_candidates.collect()
    print('Validating')
    candidates_validation(target_pl=sampled_train_candidates)
    # print('Sampled train data')
    print(sampled_train_candidates.shape)
    print(f"Memory: {sampled_train_candidates.estimated_size(unit='gb')}")
    print(sampled_train_candidates['target'].mean())

    stat_info = (
            sampled_train_candidates
                .groupby('session_id')
                .agg(
                    pl.count().alias('rec_cnt')
                    , pl.col('target').sum().alias('target_num')
                )
                .describe()
        )
    print(stat_info)
    if not debug:
        sampled_train_candidates.write_parquet(
            # os.path.join(ranker_train_data_dir, 'train.parquet')
            file_name
        )
# del sampled_train_candidates

  0%|          | 0/20 [00:00<?, ?it/s]

../data/rank_train_data_task1_v10/train/part_1.parquet
Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_train_w2v_task1_v10_True_top100.parquet
../data/candidates/task1_train_nic_task1_v10_True_top100.parquet
../data/candidates/task1_train_nfi_task1_v10_True_top100.parquet
../data/candidates/task1_train_co_visit_task1_v10_True_top300.parquet
Original negative num: 8321045
Validating
(8377674, 24)
Memory: 1.404417060315609
0.005178167591625074
shape: (7, 4)
┌────────────┬───────────────┬────────────┬────────────┐
│ describe   ┆ session_id    ┆ rec_cnt    ┆ target_num │
│ ---        ┆ ---           ┆ ---        ┆ ---        │
│ str        ┆ f64           ┆ f64        ┆ f64        │
╞════════════╪═══════════════╪════════════╪════════════╡
│ count      ┆ 43381.0       ┆ 43381.0    ┆ 43381.0    │
│ null_count ┆ 0.0           ┆ 0.0        ┆ 0.0        │
│ mean       ┆ 1.6246e6      ┆ 193.118508 ┆ 1.0        │
│ std        ┆ 936145.700739 ┆ 105.065401 ┆ 0.0        │
│ min    

  5%|▌         | 1/20 [05:34<1:45:51, 334.30s/it]

../data/rank_train_data_task1_v10/train/part_2.parquet
Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_train_w2v_task1_v10_True_top100.parquet
../data/candidates/task1_train_nic_task1_v10_True_top100.parquet
../data/candidates/task1_train_nfi_task1_v10_True_top100.parquet
../data/candidates/task1_train_co_visit_task1_v10_True_top300.parquet
Original negative num: 8292434
Validating
(8348848, 24)
Memory: 1.3995847254991531
0.005180834529506346
shape: (7, 4)
┌────────────┬───────────────┬────────────┬────────────┐
│ describe   ┆ session_id    ┆ rec_cnt    ┆ target_num │
│ ---        ┆ ---           ┆ ---        ┆ ---        │
│ str        ┆ f64           ┆ f64        ┆ f64        │
╞════════════╪═══════════════╪════════════╪════════════╡
│ count      ┆ 43254.0       ┆ 43254.0    ┆ 43254.0    │
│ null_count ┆ 0.0           ┆ 0.0        ┆ 0.0        │
│ mean       ┆ 1.6262e6      ┆ 193.019096 ┆ 1.0        │
│ std        ┆ 933915.184793 ┆ 105.068708 ┆ 0.0        │
│ min   

 10%|█         | 2/20 [11:41<1:46:10, 353.93s/it]

...

../data/rank_train_data_task1_v10/train/part_7.parquet
Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_train_w2v_task1_v10_True_top100.parquet
../data/candidates/task1_train_nic_task1_v10_True_top100.parquet
../data/candidates/task1_train_nfi_task1_v10_True_top100.parquet
../data/candidates/task1_train_co_visit_task1_v10_True_top300.parquet
Original negative num: 8302573
Validating
(8358971, 24)
Memory: 1.401281725615263
0.0051675020765115705
shape: (7, 4)
┌────────────┬───────────────┬────────────┬────────────┐
│ describe   ┆ session_id    ┆ rec_cnt    ┆ target_num │
│ ---        ┆ ---           ┆ ---        ┆ ---        │
│ str        ┆ f64           ┆ f64        ┆ f64        │
╞════════════╪═══════════════╪════════════╪════════════╡
│ count      ┆ 43195.0       ┆ 43195.0    ┆ 43195.0    │
│ null_count ┆ 0.0           ┆ 0.0        ┆ 0.0        │
│ mean       ┆ 1.6109e6      ┆ 193.517097 ┆ 1.0        │
│ std        ┆ 934091.197921 ┆ 105.155147 ┆ 0.0        │
│ min   

 35%|███▌      | 7/20 [41:03<1:18:34, 362.66s/it]

../data/rank_train_data_task1_v10/train/part_8.parquet
Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_train_w2v_task1_v10_True_top100.parquet
../data/candidates/task1_train_nic_task1_v10_True_top100.parquet
../data/candidates/task1_train_nfi_task1_v10_True_top100.parquet
../data/candidates/task1_train_co_visit_task1_v10_True_top300.parquet
Original negative num: 8326640
Validating
(8382979, 24)
Memory: 1.4053063802421093
0.005167733332029103
shape: (7, 4)
┌────────────┬───────────────┬────────────┬────────────┐
│ describe   ┆ session_id    ┆ rec_cnt    ┆ target_num │
│ ---        ┆ ---           ┆ ---        ┆ ---        │
│ str        ┆ f64           ┆ f64        ┆ f64        │
╞════════════╪═══════════════╪════════════╪════════════╡
│ count      ┆ 43321.0       ┆ 43321.0    ┆ 43321.0    │
│ null_count ┆ 0.0           ┆ 0.0        ┆ 0.0        │
│ mean       ┆ 1.6177e6      ┆ 193.508437 ┆ 1.0        │
│ std        ┆ 932126.065498 ┆ 105.429914 ┆ 0.0        │
│ min   

 40%|████      | 8/20 [47:13<1:13:01, 365.14s/it]

../data/rank_train_data_task1_v10/train/part_9.parquet
Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_train_w2v_task1_v10_True_top100.parquet
../data/candidates/task1_train_nic_task1_v10_True_top100.parquet
../data/candidates/task1_train_nfi_task1_v10_True_top100.parquet
../data/candidates/task1_train_co_visit_task1_v10_True_top300.parquet


In [ ]:
# if debug:
#     train_parts = 1
# for part_index in tqdm(range(train_parts)):

part_index = 0
file_name = os.path.join(train_dir, f"part_{part_index+1}.parquet")
print(file_name)
# print(file_name)
train_cg_pl = get_all_candidates(data_type='train', 
                                 part=part_index+1,
                                 # original_file_name='../data/raw_data_session_id/sess'
                                 w2v_model_version=f'w2v_{task}_{version}'
                                )
sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', 
                                       negative_sampling=0.1
                            , nic_model=nic_model
                         , nfi_model=nfi_model
                          , co_visit_model=co_visit_model
                                      )
sampled_train_candidates = sampled_train_candidates.collect()
print('Validating')
candidates_validation(target_pl=sampled_train_candidates)
# print('Sampled train data')
print(sampled_train_candidates.shape)
print(f"Memory: {sampled_train_candidates.estimated_size(unit='gb')}")
print(sampled_train_candidates['target'].mean())

stat_info = (
        sampled_train_candidates
            .groupby('session_id')
            .agg(
                pl.count().alias('rec_cnt')
                , pl.col('target').sum().alias('target_num')
            )
            .describe()
    )
print(stat_info)
# if not debug:
#     sampled_train_candidates.write_parquet(
#         # os.path.join(ranker_train_data_dir, 'train.parquet')
#         file_name
#     )
# del sampled_train_candidates

../data/rank_train_data_task1_v10/train/part_1.parquet
Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_train_w2v_task1_v10_True_top100.parquet
../data/candidates/task1_train_nic_task1_v10_True_top100.parquet
../data/candidates/task1_train_nfi_task1_v10_True_top100.parquet
../data/candidates/task1_train_co_visit_task1_v10_True_top300.parquet
Original negative num: 8321045


In [103]:
rec_df = train_cg_pl.select(['rec_num', 'strategy_num']).collect().to_pandas()


In [100]:
rec_df['rec_num'].describe([0.1*i for i in range(10)])

count    72487.000000
mean       177.461324
std        109.100657
min          1.000000
0%           1.000000
10%         33.000000
20%        106.000000
30%        116.000000
40%        128.000000
50%        146.000000
60%        172.000000
70%        212.000000
80%        277.000000
90%        364.000000
max        478.000000
Name: rec_num, dtype: float64

In [102]:
1/30

0.03333333333333333

<100 -> same
<150 -> 1/10 

In [ ]:
# sampled_train_candidates.head()

In [ ]:
os.path.join(ranker_train_data_dir, 'train.parquet')

In [203]:
# ! rm {file_name}

# Debug 

In [205]:
stat_info = (
        sampled_train_candidates
            .groupby('session_id')
            .agg(
                pl.count().alias('rec_cnt')
                , pl.col('target').sum().alias('target_num')
            )
    ).to_pandas()

In [211]:
stat_info.describe([0.001*i for i in range(10)])

,session_id,rec_cnt,target_num
count,1.036500e+05,103650.000000,103650.0
mean,3.439392e+06,147.781061,1.0
std,9.599537e+04,68.051765,0.0
min,3.272717e+06,1.000000,1.0
0%,3.272717e+06,1.000000,1.0
0.1%,3.273067e+06,1.000000,1.0
0.2%,3.273447e+06,1.000000,1.0
0.3%,3.273800e+06,2.000000,1.0
0.4%,3.274143e+06,2.000000,1.0
0.5%,3.274497e+06,2.000000,1.0


In [214]:
train_cg_pl.select('rec_num').collect().describe()

describe,rec_num
str,f64
"""count""",148830.0
"""null_count""",0.0
"""mean""",140.949506
"""std""",68.54101
"""min""",1.0
"""max""",430.0
"""median""",122.0


In [215]:
train_cg_pl.filter(pl.col('rec_num')==1).collect()

session_id,next_item_prediction,w2v_weight,whether_w2v,whether_nic,whether_nfi,whether_covisit,strategy_num,next_item,locale,prev_length,last_prev_item,rec_num
i64,list[str],list[f64],list[i64],list[i64],list[i64],list[i64],list[i64],str,str,u32,str,u32
3448064,"[""B0B8YD9WFN""]",[1.0],[0],[0],[0],[1],[1],"""B0B8YD9WFN""","""FR""",2,"""B0B8YHGF2B""",1
3509248,"[""B0B86J3YMK""]",[1.0],[0],[1],[1],[1],[3],"""B0B86J3YMK""","""IT""",2,"""B0B86FQQ26""",1
3589408,"[""B0B58X7BG6""]",[1.0],[0],[0],[0],[1],[1],"""B0B58X7BG6""","""IT""",2,"""B0B93JWTQR""",1
3521824,"[""B08N6BLCDX""]",[1.0],[0],[0],[0],[1],[1],"""B08M3MCK8J""","""IT""",3,"""B09JRKD5S6""",1
3447744,"[""229016545X""]",[1.0],[0],[0],[0],[1],[1],"""2701164613""","""FR""",2,"""2266294563""",1
3545952,"[""B07Y5KS92H""]",[1.0],[0],[0],[0],[1],[1],"""B0BGK9WCD5""","""IT""",4,"""B08MBRT26Y""",1
3444480,"[""B06XPD3BP2""]",[1.0],[0],[0],[0],[1],[1],"""B0B82D3VXK""","""FR""",2,"""B07N2HHSNZ""",1
3318400,"[""B0816JTCKG""]",[1.0],[0],[0],[0],[1],[1],"""B0816JTCKG""","""ES""",3,"""B07VF87RFD""",1
3513984,"[""B08LN1V74X""]",[1.0],[0],[1],[1],[1],[3],"""B09TRQZDNW""","""IT""",4,"""B09T8JMWJ6""",1


In [213]:
train_cg_pl.head().collect()

session_id,next_item_prediction,w2v_weight,whether_w2v,whether_nic,whether_nfi,whether_covisit,strategy_num,next_item,locale,prev_length,last_prev_item,rec_num
i64,list[str],list[f64],list[i64],list[i64],list[i64],list[i64],list[i64],str,str,u32,str,u32
3321184,"[""B08S73SPCK"", ""2376712882"", … ""B09T9QQ4BC""]","[0.0, 0.41, … 0.4]","[0, 1, … 1]","[0, 0, … 0]","[1, 0, … 0]","[1, 0, … 0]","[2, 1, … 1]","""B099VZ6Y7Z""","""ES""",5,"""B099WD5CS9""",126
3418016,"[""B0792QLDCW"", ""B00UW8QUOK"", … ""B07H9WS3Y7""]","[0.6, 0.15, … 0.0]","[1, 1, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 1]","[1, 1, … 1]","""B005C0RJKM""","""FR""",5,"""B09Q956R8R""",227
3540992,"[""B07M7DMTVH"", ""B07W7CWLGD"", … ""B01702H4I6""]","[0.18, 0.72, … 0.94]","[1, 1, … 1]","[0, 0, … 0]","[0, 0, … 1]","[1, 0, … 1]","[2, 1, … 3]","""B093WYS6B7""","""IT""",3,"""B09HY2B6YP""",125
3407264,"[""2226328432"", ""B082YPRHN7"", … ""B08Q7GJBHH""]","[0.95, 0.26, … 0.9]","[1, 1, … 1]","[0, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[1, 1, … 1]","""B08P2YC9T1""","""FR""",2,"""B08P2ZYMV9""",131
3284288,"[""B0BGLBDL6T"", ""B08ZJMPY2T"", … ""B09QS7K6CP""]","[0.8, 0.0, … 0.0]","[1, 0, … 0]","[0, 0, … 0]","[0, 0, … 0]","[0, 1, … 1]","[1, 1, … 1]","""B07YGZ7JD5""","""ES""",3,"""B07ZG8W8B4""",194


# Slow speed -> **7 hours to finished all of the data**



```
../data/rank_train_data_v2/train/part_1.parquet
Validating
(28566071, 11)
Memory: 2.23475506529212
0.005130737090165463
shape: (7, 4)
┌────────────┬──────────────┬───────────┬────────────┐
│ describe   ┆ session_id   ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---          ┆ ---       ┆ ---        │
│ str        ┆ f64          ┆ f64       ┆ f64        │
╞════════════╪══════════════╪═══════════╪════════════╡
│ count      ┆ 146565.0     ┆ 146565.0  ┆ 146565.0   │
│ null_count ┆ 0.0          ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 81754.567209 ┆ 194.90377 ┆ 1.0        │
│ std        ┆ 47232.319874 ┆ 96.523984 ┆ 0.0        │
│ min        ┆ 0.0          ┆ 80.0      ┆ 1.0        │
│ max        ┆ 163580.0     ┆ 468.0     ┆ 1.0        │
│ median     ┆ 81763.0      ┆ 154.0     ┆ 1.0        │
└────────────┴──────────────┴───────────┴────────────┘
  5%|▌         | 1/20 [22:21<7:04:44, 1341.26s/it]

../data/rank_train_data_v2/train/part_2.parquet

```

# Eval

In [ ]:
# ! ls {ranker_train_data_dir}

In [ ]:
# ! rm {ranker_train_data_dir}/*

In [ ]:
# pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test.parquet')).collect().shape

In [ ]:
# pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet')).collect().shape

# Case study 

In [42]:
# train_candidate = get_all_candidates(data_type='train', part)

In [43]:
w2v, nic, nfi, covisit = get_train_candidate()

../data/candidates/task2_train_w2v_task2_v2_True_top100
../data/candidates/task2_train_nic_task2_v2_True_top100
../data/candidates/task2_train_nfi_task2_v2_True_top100
../data/candidates/task2_train_co_visit_task2_v2_True_top300


In [44]:
session_id = 3272717

In [45]:
nic.filter(pl.col('session_id')==session_id).head()

session_id,next_item_prediction,rec_num
i64,list[str],u32
3272717,"[""B07JGW4QWX"", ""B07JH6VD7T"", … ""B00XO27QH6""]",12


In [46]:
get_all_candidates(data_type='train', part=part_index+1, 
                   detail=True
                                    )

Candidiate files: 
++++++++++++++++++++

../data/candidates/task2_train_w2v_task2_v2_True_top100.parquet
../data/candidates/task2_train_nic_task2_v2_True_top100.parquet
../data/candidates/task2_train_nfi_task2_v2_True_top100.parquet
../data/candidates/task2_train_co_visit_task2_v2_True_top300.parquet
Type: w2v
shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7518.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 91.894919 │
│ std        ┆ 27.285348 │
│ min        ┆ 0.0       │
│ max        ┆ 100.0     │
│ median     ┆ 100.0     │
└────────────┴───────────┘
Type: nic
shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7518.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 15.844507 │
│ std        ┆ 19.049389 │
│ min        ┆ 0.0       │
│ max        ┆ 100.0     │
│ 

In [48]:
for part_index in tqdm(range(train_parts)):
    print()
    file_name = os.path.join(train_dir, f"part_{part_index+1}.parquet")
    print(file_name)
    train_cg_pl = get_all_candidates(data_type='train', part=part_index+1, 
                   detail=True
                                    )
    # length = train_cg_pl.filter(pl.col('session_id')==session_id).collect().shape[0]
    # if length > 0:
    #     single_case = train_cg_pl.filter(pl.col('session_id')==session_id).collect()
    # sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', 
    #                                        negative_sampling=0.1
    #                             , nic_model=nic_model
    #                          , nfi_model=nfi_model
    #                           , co_visit_model=co_visit_model
    #                                       )
    

  5%|▌         | 1/20 [00:00<00:02,  7.46it/s]


../data/rank_train_data_task2_v2/train/part_1.parquet
Candidiate files: 
++++++++++++++++++++

../data/candidates/task2_train_w2v_task2_v2_True_top100.parquet
../data/candidates/task2_train_nic_task2_v2_True_top100.parquet
../data/candidates/task2_train_nfi_task2_v2_True_top100.parquet
../data/candidates/task2_train_co_visit_task2_v2_True_top300.parquet
Type: w2v
shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7502.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 88.984004 │
│ std        ┆ 31.302456 │
│ min        ┆ 0.0       │
│ max        ┆ 100.0     │
│ median     ┆ 100.0     │
└────────────┴───────────┘
Type: nic
shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7502.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 15.760331 │
│ std        ┆ 19.150973 │
│

 15%|█▌        | 3/20 [00:00<00:02,  7.40it/s]

shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7502.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 15.618902 │
│ std        ┆ 18.618445 │
│ min        ┆ 0.0       │
│ max        ┆ 100.0     │
│ median     ┆ 9.0       │
└────────────┴───────────┘
Type: nfi
shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7502.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 34.716476 │
│ std        ┆ 32.571511 │
│ min        ┆ 0.0       │
│ max        ┆ 100.0     │
│ median     ┆ 23.0      │
└────────────┴───────────┘
Type: covisit
shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7502.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 59.812583 │
│ std        ┆ 6

 20%|██        | 4/20 [00:00<00:02,  7.52it/s]

...

shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7502.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 62.161557 │
│ std        ┆ 69.59397  │
│ min        ┆ 0.0       │
│ max        ┆ 300.0     │
│ median     ┆ 36.0      │
└────────────┴───────────┘



../data/rank_train_data_task2_v2/train/part_17.parquet
Candidiate files: 
++++++++++++++++++++

../data/candidates/task2_train_w2v_task2_v2_True_top100.parquet
../data/candidates/task2_train_nic_task2_v2_True_top100.parquet
../data/candidates/task2_train_nfi_task2_v2_True_top100.parquet
../data/candidates/task2_train_co_visit_task2_v2_True_top300.parquet
Type: w2v
shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7502.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 92.730205 │
│ std        ┆ 25.956277 │
│ min   

 95%|█████████▌| 19/20 [00:02<00:00,  7.40it/s]

shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7502.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 16.323914 │
│ std        ┆ 19.080199 │
│ min        ┆ 0.0       │
│ max        ┆ 100.0     │
│ median     ┆ 9.0       │
└────────────┴───────────┘
Type: nfi
shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7502.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 35.619302 │
│ std        ┆ 32.429902 │
│ min        ┆ 0.0       │
│ max        ┆ 100.0     │
│ median     ┆ 24.0      │
└────────────┴───────────┘
Type: covisit
shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7502.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 61.473207 │
│ std        ┆ 6

100%|██████████| 20/20 [00:02<00:00,  7.45it/s]

shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7518.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 91.894919 │
│ std        ┆ 27.285348 │
│ min        ┆ 0.0       │
│ max        ┆ 100.0     │
│ median     ┆ 100.0     │
└────────────┴───────────┘
Type: nic
shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7518.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 15.844507 │
│ std        ┆ 19.049389 │
│ min        ┆ 0.0       │
│ max        ┆ 100.0     │
│ median     ┆ 9.0       │
└────────────┴───────────┘
Type: nfi
shape: (7, 2)
┌────────────┬───────────┐
│ describe   ┆ rec_cnt   │
│ ---        ┆ ---       │
│ str        ┆ f64       │
╞════════════╪═══════════╡
│ count      ┆ 7518.0    │
│ null_count ┆ 0.0       │
│ mean       ┆ 34.265629 │
│ std        ┆ 32.04

In [29]:
single_case

session_id,next_item_prediction,w2v_weight,next_item,locale,prev_length,last_prev_item
i64,list[str],list[f64],str,str,u32,str
3272717,"[""B0875NB89J"", ""B0B42FLYF6"", … ""B098FJ6LXB""]","[0.0, 0.87, … 0.0]","""B07JFPYN5P""","""ES""",2,"""B085VCXHXL"""


In [30]:
# single_case.'next_item_prediction']

next_item_prediction
list[str]
"[""B0875NB89J"", ""B0B42FLYF6"", … ""B098FJ6LXB""]"


In [65]:
# search_id = 9184

In [48]:
# data_type = 'train'
# part = 0 +1 
# w2v_file = os.path.join(candidate_path, 
#                        candidate_file_name.format(
#                 task=task
#             , data_type=data_type
#             , model_version=w2v_model_version
#             , model_for_eval=model_for_eval
#             , topn=w2v_topn
#                        ))
# nic_file = os.path.join(candidate_path,
#             candidate_file_name.format(
#                 task=task
#                 , data_type=data_type
#                 , model_version=nic_model_version
#                 , model_for_eval=model_for_eval
#                 , topn=nic_topn
#                        ))
# nfi_file = os.path.join(candidate_path,
#         candidate_file_name.format(
#             task=task
#             , data_type=data_type
#             , model_version=nfi_model_version
#             , model_for_eval=model_for_eval
#             , topn=nfi_topn
#                    ))
# covisit_file = os.path.join(candidate_path,
#         candidate_file_name.format(
#             task=task
#             , data_type=data_type
#             , model_version=covisit_model_version
#             , model_for_eval=model_for_eval
#             , topn=covisit_topn
#                    ))
# if data_type == 'train':
#     assert part is not None
#     w2v_file = os.path.join(w2v_file.replace('.parquet', ''), f"part_{part}.parquet")
#     nic_file = os.path.join(nic_file.replace('.parquet', ''), f"part_{part}.parquet")
#     nfi_file = os.path.join(nfi_file.replace('.parquet', ''), f"part_{part}.parquet")
#     covisit_file = os.path.join(covisit_file.replace('.parquet', ''), f"part_{part}.parquet")

# if data_type == 'test':
#     original_file_name = f"sessions_{data_type}_{task}.parquet"
# else:
#     original_file_name = f"sessions_{data_type}.parquet"
# original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))


# #     def get_vector(prev_items):
# #         prev_items_vec = np.array([w2vec.wv[ele] for ele in prev_items]).mean(axis=0)
# #         return prev_items_vec

# #     original_pl = (
# #             original_pl.with_columns(
# #                 pl.col('prev_items').apply(lambda x: get_vector(x)).alias('all_previous_items_vec')
# #             )
# #         )

# # get w2v weight
# w2v_pl = w2v_pl.with_columns(pl.col('next_item_prediction').arr.lengths().alias('rec_num')).with_columns(
#     pl.col('rec_num').apply(lambda x: list(range(x, 0, -1))).alias('w2v_weight')

# )
# # print(w2v_pl.head().collect())
# get_w2v_weight = pl.element().rank()*0
# nic_pl = nic_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                             # parallel=True
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int64))
# )
# nfi_pl = nfi_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int64))
# )
# covisit_pl = covisit_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int64))
# )
# cols = ['session_id', 'next_item_prediction', 
#         'w2v_weight'
#        ]


In [57]:
# nic_rec = nic_pl.filter(pl.col('session_id')==search_id).select('next_item_prediction').collect().to_series().to_list()[0]
# w2v_rec = w2v_pl.filter(pl.col('session_id')==search_id).select('next_item_prediction').collect().to_series().to_list()[0]